In [1]:
import numpy as np

class ImprovedMatrixFactorizationWithBias:
    def __init__(self,
                 alpha=0.001,
                 iterations=20,
                 num_latent=100,
                 lambda_reg=0.02,
                 num_users=943,
                 num_items=1682,
                 early_stopping=5):
        """
        Initialize the model with hyperparameters and configuration.
        """
        self.alpha = alpha  # Learning rate
        self.iterations = iterations  # Number of iterations for training
        self.num_latent = num_latent  # Number of latent factors
        self.lambda_reg = lambda_reg  # Regularization parameter
        self.num_users = num_users  # Total number of users
        self.num_items = num_items  # Total number of items
        self.early_stopping = early_stopping  # Patience for early stopping

    def fit(self, train_data, validation_data=None):
        """
        Train the model using training data with optional validation data.
        """
        self.train = train_data

        # Initialize latent factors P and Q using Xavier/Glorot initialization
        limit = np.sqrt(6 / (self.num_users + self.num_items))
        self.P = np.random.uniform(-limit, limit, (self.num_users, self.num_latent))
        self.Q = np.random.uniform(-limit, limit, (self.num_items, self.num_latent))

        # Initialize biases with small random values
        self.bu = np.random.normal(0, 0.01, self.num_users)
        self.bi = np.random.normal(0, 0.01, self.num_items)
        self.global_bias = np.mean([r for _, _, r in train_data])  # Mean rating as global bias

        best_rmse = float('inf')  # Best RMSE on validation set
        patience = self.early_stopping  # Remaining patience for early stopping
        rmse_history = []  # To store RMSE values for each iteration

        for iteration in range(self.iterations):
            np.random.shuffle(self.train)  # Shuffle training data
            squared_error = 0  # Accumulate squared errors
            count = 0  # Counter for processed ratings

            # Mini-batch gradient descent
            batch_size = 1000
            for i in range(0, len(self.train), batch_size):
                batch = self.train[i:i + batch_size]

                for user_id, item_id, rating in batch:
                    # Predict the rating for the given user-item pair
                    pred = self._predict_single(user_id, item_id)
                    error = rating - pred

                    squared_error += error ** 2
                    count += 1

                    # Gradient descent with momentum initialization
                    if not hasattr(self, 'P_momentum'):
                        self.P_momentum = np.zeros_like(self.P)
                        self.Q_momentum = np.zeros_like(self.Q)
                        self.bu_momentum = np.zeros_like(self.bu)
                        self.bi_momentum = np.zeros_like(self.bi)

                    momentum = 0.9  # Momentum factor

                    # Update latent factors P
                    grad_P = 2 * error * self.Q[item_id] - self.lambda_reg * self.P[user_id]
                    self.P_momentum[user_id] = momentum * self.P_momentum[user_id] + self.alpha * grad_P
                    self.P[user_id] += self.P_momentum[user_id]

                    # Update latent factors Q
                    grad_Q = 2 * error * self.P[user_id] - self.lambda_reg * self.Q[item_id]
                    self.Q_momentum[item_id] = momentum * self.Q_momentum[item_id] + self.alpha * grad_Q
                    self.Q[item_id] += self.Q_momentum[item_id]

                    # Update user bias bu
                    grad_bu = 2 * error - self.lambda_reg * self.bu[user_id]
                    self.bu_momentum[user_id] = momentum * self.bu_momentum[user_id] + self.alpha * grad_bu
                    self.bu[user_id] += self.bu_momentum[user_id]

                    # Update item bias bi
                    grad_bi = 2 * error - self.lambda_reg * self.bi[item_id]
                    self.bi_momentum[item_id] = momentum * self.bi_momentum[item_id] + self.alpha * grad_bi
                    self.bi[item_id] += self.bi_momentum[item_id]

            # Compute RMSE for training data
            rmse = np.sqrt(squared_error / count)
            rmse_history.append(rmse)

            # Validation check
            if validation_data is not None:
                val_rmse = np.sqrt(np.mean([(r - self._predict_single(u, i)) ** 2
                                            for u, i, r in validation_data]))
                print(f"Iteration {iteration + 1}/{self.iterations}, Train RMSE: {rmse:.4f}, Val RMSE: {val_rmse:.4f}")

                if val_rmse < best_rmse:
                    # Save the best parameters
                    best_rmse = val_rmse
                    patience = self.early_stopping
                    self.best_P = self.P.copy()
                    self.best_Q = self.Q.copy()
                    self.best_bu = self.bu.copy()
                    self.best_bi = self.bi.copy()
                else:
                    patience -= 1
                    if patience == 0:
                        print("Early stopping!")
                        # Restore the best parameters
                        self.P = self.best_P
                        self.Q = self.best_Q
                        self.bu = self.best_bu
                        self.bi = self.best_bi
                        break
            else:
                print(f"Iteration {iteration + 1}/{self.iterations}, RMSE: {rmse:.4f}")

            # Decay learning rate
            self.alpha *= 0.995

        self.rmse_history = rmse_history  # Save RMSE history

    def _predict_single(self, user_id, item_id):
        """
        Predict the rating for a single user-item pair.
        """
        pred = (self.Q[item_id] @ self.P[user_id] +
                self.bu[user_id] +
                self.bi[item_id] +
                self.global_bias)
        # Clip predictions to be within the allowed range
        return np.clip(pred, 1, 5)

    def predict(self, test_data):
        """
        Predict ratings for a set of test user-item pairs.
        """
        return [self._predict_single(u, i) for u, i, _ in test_data]


**Functions**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Prepare data for training
def prepare_data(df_data):
    # Convert the data into a list of tuples (userId, movieId, rating)
    data = [(row.userId, row.movieId, row.rating) for idx, row in df_data.iterrows()]
    # Split the data into training and test sets
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    return train_data, test_data

In [3]:
def predict_for_user(clf, user_id, movie_ids):
    """
    Generates a sorted list of predictions for a specific user.

    Args:
        clf: Instance of ImprovedMatrixFactorizationWithBias.
        user_id: ID of the user for whom predictions are being made.
        movie_ids: List of movie IDs to consider for predictions.

    Returns:
        List of tuples (movie_id, predicted_rating) sorted in descending order of ratings.
    """
    predictions = []
    # Loop through the provided movie IDs
    for movie_id in movie_ids:
        # Predict the rating for the user-movie pair using the model's single prediction method
        pred = clf._predict_single(user_id, movie_id)
        predictions.append((movie_id, pred))  # Store the movie ID and its predicted rating as a tuple

    # Sort the predictions by rating in descending order
    return sorted(predictions, key=lambda x: x[1], reverse=True)

In [4]:
def calculate_all_metrics(model, test_data, n_movies, k=10):
    """
    Calculates Recall@K, Precision@K, NDCG@K, Gini Index, and Coverage for a recommendation system.

    Args:
        model: Recommendation model (e.g., ImprovedMatrixFactorizationWithBias).
        test_data: Test dataset in the format (user_id, movie_id, rating).
        n_movies: Total number of movies in the dataset.
        k: Number of recommendations to consider.

    Returns:
        metrics: Dictionary containing all calculated metrics.
    """
    # Group true items (movies with high ratings) for each user
    user_true_items = {}
    for user, movie, rating in test_data:
        if user not in user_true_items:
            user_true_items[user] = []
        if rating >= 3.5:  # Only consider high ratings as "true" items
            user_true_items[user].append(movie)

    recommendations = {}
    item_counts = np.zeros(n_movies)  # For calculating Coverage and Gini Index

    recall_list = []
    precision_list = []
    ndcg_list = []

    for user, true_items in user_true_items.items():
        # Generate recommendations for the user
        recommended_items = predict_for_user(model, user, range(n_movies))
        top_k_items = [item[0] for item in recommended_items[:k]]  # Extract top-K item IDs
        recommendations[user] = top_k_items

        # Update the frequency of recommended items
        for item in top_k_items:
            item_counts[item] += 1

        # Calculate user-specific metrics
        tp = len(set(top_k_items) & set(true_items))  # True positives

        # Recall@K: Proportion of true items retrieved in the top-K recommendations
        recall = tp / len(true_items) if true_items else 0
        recall_list.append(recall)

        # Precision@K: Proportion of top-K recommendations that are true items
        precision = tp / k
        precision_list.append(precision)

        # NDCG@K: Normalized Discounted Cumulative Gain
        dcg = 0
        idcg = 0
        for i, item in enumerate(top_k_items):
            if item in true_items:
                dcg += 1 / np.log2(i + 2)  # Discounted gain
        for i in range(min(len(true_items), k)):
            idcg += 1 / np.log2(i + 2)  # Ideal discounted gain
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcg_list.append(ndcg)

    # Coverage: Proportion of items recommended at least once
    coverage = np.sum(item_counts > 0) / n_movies

    # Gini Index: Measure of inequality in item recommendation frequencies
    sorted_counts = np.sort(item_counts)  # Sort item frequencies
    n = len(sorted_counts)
    gini = 1 - (2 / (n - 1)) * np.sum((n - np.arange(1, n + 1)) * sorted_counts) / np.sum(sorted_counts)

    # Compute the average metrics across all users
    metrics = {
        "Recall@K": np.mean(recall_list),
        "Precision@K": np.mean(precision_list),
        "NDCG@K": np.mean(ndcg_list),
        "Gini Index": gini,
        "Coverage": coverage
    }

    return metrics

## **1M Dataset**

In [5]:
df_ratings_path = "" # insert here the path of 'ratings.csv' dataset

# Load the ratings dataset
df_ratings = pd.read_csv(df_ratings_path, delimiter='::', header=None)
df_ratings.columns = ['userId', 'movieId', 'rating', 'timestamp']

# Subtract 1 from userId and movieId to make them zero-based
df_ratings['userId'] = df_ratings['userId'] - 1
df_ratings['movieId'] = df_ratings['movieId'] - 1

# -- Movie IDs are not sequential, so they need to be remapped
# Get unique movie IDs and sort them
unique_movieIds = sorted(df_ratings['movieId'].unique())
# Create a dictionary mapping original movie IDs to new sequential IDs
movieId_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_movieIds)}
# Apply the mapping to the 'movieId' column
df_ratings['movieId'] = df_ratings['movieId'].map(movieId_mapping)

n_users =  len(df_ratings['userId'].unique())  # Total number of users
n_movies = len(df_ratings['movieId'].unique())  # Total number of unique movies
print(n_users, n_movies)

/tmp/ipykernel_70115/207279230.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ratings = pd.read_csv(df_ratings_path, delimiter='::', header=None)


6040 3706


In [6]:
train_data, test_data = prepare_data(df_ratings)

model = ImprovedMatrixFactorizationWithBias(
    alpha=0.001,           # Learning rate for gradient descent
    iterations=20,         # Number of training epochs
    num_latent=100,        # Dimensionality of latent factors
    lambda_reg=0.02,       # Regularization parameter to prevent overfitting
    num_users=n_users,     # Total number of users in the dataset
    num_items=n_movies     # Total number of movies in the dataset
)
model.fit(train_data, test_data)

metrics = calculate_all_metrics(model, test_data, n_movies, k=10)
print(metrics)

Iteration 1/20, Train RMSE: 0.9458, Val RMSE: 0.9219
Iteration 2/20, Train RMSE: 0.9105, Val RMSE: 0.9163
Iteration 3/20, Train RMSE: 0.9016, Val RMSE: 0.9058
Iteration 4/20, Train RMSE: 0.8802, Val RMSE: 0.8877
Iteration 5/20, Train RMSE: 0.8498, Val RMSE: 0.8719
Iteration 6/20, Train RMSE: 0.8119, Val RMSE: 0.8610
Iteration 7/20, Train RMSE: 0.7674, Val RMSE: 0.8556
Iteration 8/20, Train RMSE: 0.7178, Val RMSE: 0.8573
Iteration 9/20, Train RMSE: 0.6688, Val RMSE: 0.8640
Iteration 10/20, Train RMSE: 0.6242, Val RMSE: 0.8731
Iteration 11/20, Train RMSE: 0.5853, Val RMSE: 0.8830
Iteration 12/20, Train RMSE: 0.5528, Val RMSE: 0.8931
Early stopping!
{'Recall@K': 0.04112499771117983, 'Precision@K': 0.05771778734680358, 'NDCG@K': 0.06536283639007044, 'Gini Index': 0.9835295222028368, 'Coverage': 0.13437668645439826}


# **100K dataset**

In [ ]:
df_ratings_path = "" # insert here the path of 'ratings.csv' dataset

# Load the ratings dataset
df_ratings = pd.read_csv(df_ratings_path, delimiter='\t', header=None)
df_ratings.columns = ['userId', 'movieId', 'rating', 'timestamp']

# Preprocessing
df_ratings['userId'] = df_ratings['userId'] - 1
df_ratings['movieId'] = df_ratings['movieId'] - 1

n_users =  len(df_ratings['userId'].unique())  # Total number of users
n_movies = len(df_ratings['movieId'].unique())  # Total number of unique movies
print(n_users, n_movies)

943 1682


In [8]:
train_data, test_data = prepare_data(df_ratings)

model = ImprovedMatrixFactorizationWithBias(
    alpha=0.001,           # Learning rate for gradient descent
    iterations=20,         # Number of training epochs
    num_latent=100,        # Dimensionality of latent factors
    lambda_reg=0.02,       # Regularization parameter to prevent overfitting
    num_users=n_users,     # Total number of users in the dataset
    num_items=n_movies     # Total number of movies in the dataset
)
model.fit(train_data, test_data)

metrics = calculate_all_metrics(model, test_data, n_movies, k=10)
print(metrics)

Iteration 1/20, Train RMSE: 1.0071, Val RMSE: 0.9622
Iteration 2/20, Train RMSE: 0.9465, Val RMSE: 0.9474
Iteration 3/20, Train RMSE: 0.9302, Val RMSE: 0.9431
Iteration 4/20, Train RMSE: 0.9183, Val RMSE: 0.9396
Iteration 5/20, Train RMSE: 0.9015, Val RMSE: 0.9345
Iteration 6/20, Train RMSE: 0.8747, Val RMSE: 0.9274
Iteration 7/20, Train RMSE: 0.8356, Val RMSE: 0.9211
Iteration 8/20, Train RMSE: 0.7877, Val RMSE: 0.9169
Iteration 9/20, Train RMSE: 0.7332, Val RMSE: 0.9161
Iteration 10/20, Train RMSE: 0.6752, Val RMSE: 0.9182
Iteration 11/20, Train RMSE: 0.6180, Val RMSE: 0.9226
Iteration 12/20, Train RMSE: 0.5625, Val RMSE: 0.9287
Iteration 13/20, Train RMSE: 0.5122, Val RMSE: 0.9345
Iteration 14/20, Train RMSE: 0.4663, Val RMSE: 0.9409
Early stopping!
{'Recall@K': 0.0375973705215825, 'Precision@K': 0.04436170212765958, 'NDCG@K': 0.0489784999091515, 'Gini Index': 0.9776633715999848, 'Coverage': 0.11831153388822829}
